#Google Colab Webcam

---

This notebook will go through the problem of working with a webcam on Google Colab. I'm going to show how to access the local machine's webcam to take the images or videos.

Live streaming by the webcam is something we can find in machine learning problems such as face detection or object detection. But the implementation for the google Colab needs some complex javascript integration. 

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

## Capture image

In the following you can find the code needed to capture a photo from your local machine webcam.

In [ ]:
# Javascript code for capture a photo
js_takePhoto = Javascript('''

    async function takePhoto(quality = 1.0) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = await navigator.mediaDevices.getUserMedia({video: true});
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      video.srcObject.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    
''')

In [ ]:
def capture(name='photo.jpg', quality=1.0):
  display(js_takePhoto)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(name, 'wb') as f:
    f.write(binary)
  return name

In [ ]:
try:
  filename = capture()
  print('Saved to {}'.format(filename))
  
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

## Stream Video
In the following you can find the code needed to stream a live video from your local machine webcam.

In [ ]:
# Javascript code for video stream
js_video = Javascript('''

    async function stream() {

        const div = document.createElement('div');
        const stop = document.createElement('button');
        stop.textContent = 'Stop';
        div.appendChild(stop);

        const video = document.createElement('video');
        video.style.display = 'block';

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = await navigator.mediaDevices.getUserMedia({video: true});
        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        await new Promise((resolve) => stop.onclick = resolve);
        video.srcObject.getVideoTracks()[0].stop();
        div.remove();
    }  
      
''')

In [ ]:
try:
  display(js_video)
  eval_js('stream()')

except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

<IPython.core.display.Javascript object>

## Record Video


In [ ]:
js=Javascript("""
    async function recordVideo() {
        
        const options = { mimeType: "video/webm; codecs=vp9" };
        const div = document.createElement('div');
        
        const record = document.createElement('button');
        record.textContent = "Start Recording";
        record.style.padding= '10px'
        record.style.margin= '10px'
        record.style.background = "#39b6bf";
        record.style.color = "white";
        
        div.appendChild(record);

        const stop = document.createElement("button");
        stop.textContent = "It's Recording... => To Stop the recording click again and wait for saving!";
        stop.style.padding= '10px'
        stop.style.margin= '10px'
        stop.style.background = "#7b4694";
        stop.style.background = "red";
        stop.style.color = "white";
        
        const video = document.createElement('video');
        video.style.display = 'block';
        video.srcObject = await navigator.mediaDevices.getUserMedia({audio:true, video: true});

        let recorder = new MediaRecorder(video.srcObject, options);
        document.body.appendChild(div);
        div.appendChild(video);

        video.muted = true;
        await video.play();

        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        await new Promise((resolve) => record.onclick = resolve);
        recorder.start();
        record.replaceWith(stop);

        await new Promise((resolve) => stop.onclick = resolve);
        recorder.stop();
        let recorded_data = await new Promise((resolve) => recorder.ondataavailable = resolve);
        
        video.srcObject.getVideoTracks()[0].stop();
        div.remove();

        let binaryString = "";
        let bytes = new Uint8Array(await recorded_data.data.arrayBuffer());
        bytes.forEach((byte) => {
            binaryString += String.fromCharCode(byte);
        })

        return btoa(binaryString);
    }
  """)
try:
    filename = 'recording.mkv'
    display(js)
    data=eval_js('recordVideo({})')
    binary=b64decode(data)
    with open(filename,"wb") as video_file:
         video_file.write(binary)
    print(f"Finished recording video at:{filename}")
except Exception as err:
    print(str(err))

<IPython.core.display.Javascript object>

Finished recording video at:recording.mkv
